# Assignment 4.1 - Mood Detection with OpenCV

Submitted By: Labbao, Benedick D.<br>
Performed On: 03/21/2024<br>
Submitted On: 03/22/2024

Submitted To: Engr. Roman M. Richard

---

## Dataset

### Data Collection

In [7]:
from time import sleep
import cv2
import uuid
import matplotlib.pyplot as plt
import os
import numpy as np

raw_dataset_path = 'dataset/faces/'
preprocessed_dataset_path = 'dataset/preprocessed_faces/'
final_dataset_path = 'dataset/edge_detection/'

face_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_eye.xml')

def display_image(image, title=None, conversion=cv2.COLOR_BGR2RGB):
    image = cv2.cvtColor(image, conversion)
    plt.imshow(image)
    plt.xticks([])
    plt.yticks([])
    
    if title is not None:
        plt.title(title)

    plt.show()

def fix_dataset_names(directory, prefix='face_'):
    files = os.listdir(directory)
    counter = 0
    
    for file in files:
        _, ext = os.path.splitext(file)
        new_name = f"{prefix}{counter:04d}{ext}"
        print('fixing', new_name)
        
        os.rename(os.path.join(directory, file), os.path.join(directory, new_name))
        counter += 1

def generate_name(directory):
    files = os.listdir(directory)
    num_files = len(files)
    face_names = 'face_' + str(num_files).zfill(4)
    return face_names + '.png'

def capture(count=1):
    camera = cv2.VideoCapture(0)

    while (count > 0):
        ret, frame = camera.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)

        for (x,y,w,h) in faces:
            image = frame[y:(y+h), x:(x+w)]
            name = generate_name(raw_dataset_path)
            cv2.imwrite(raw_dataset_path + name, image)
            print(f'saving {name}...')

        sleep(1)
        count-=1

In [225]:
fix_dataset_names(raw_dataset_path + 'angry/', 'angry_')

fixing angry_0000.png
fixing angry_0001.png
fixing angry_0002.png
fixing angry_0003.png
fixing angry_0004.png
fixing angry_0005.png
fixing angry_0006.png
fixing angry_0007.png
fixing angry_0008.png


In [216]:
capture(20)

saving face_0004.png...
saving face_0005.png...
saving face_0006.png...
saving face_0007.png...
saving face_0008.png...
saving face_0009.png...
saving face_0010.png...
saving face_0011.png...
saving face_0012.png...
saving face_0013.png...
saving face_0014.png...
saving face_0015.png...
saving face_0016.png...
saving face_0017.png...
saving face_0018.png...
saving face_0019.png...
saving face_0020.png...
saving face_0021.png...
saving face_0022.png...
saving face_0023.png...


This will take face images at an random interval to create a dataset for facial recognition.

### Image Pre-processing

Remove background by locating eyes and extracting face region.

In [347]:
def extract_face_region(image):
    center_point = lambda p1, p2: (int((p1[0] + p2[0]) / 2)+3, int((p1[1] + p2[1]) / 2))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, 1.03, 10, 0, (40, 40))

    # Perform eye detection then find center point of left and right eyes
    eyes_center_points = []
    for (x,y,w,h) in eyes:
        eyes_center_points.append(center_point((x,y), (x+w, y+h)))
        # cv2.rectangle(gray, (x, y), (x+w, y+h), (0, 255, 0), 1)
        # display_image(gray)
        
    # Find center betweem right eye's center point and left eye's center point
    eye_center = center_point(eyes_center_points[0], eyes_center_points[1])
    x, y = eye_center

    # add offset to create a bounding box that only contains face region
    x1, y1, x2, y2 = x - 100, y + 140, x + 100, y - 60
    cv2.rectangle(image, (x1, y1), (x2, y2),  (0, 255, 0), 2)

    return image[y2:y1, x1:x2]

def resize(image, scale=0.7):
    w, h = image.shape
    new_image = cv2.resize(image, (int(w*scale), int(h*scale)))
    return new_image

In [348]:
def preprocess_images(src_dir, dst_dir):
    files = os.listdir(src_dir)
    
    for file in files:
        file_name, ext = os.path.splitext(file)
        print("Processing image:", file_name + ext)

        image = cv2.imread(src_dir + file_name + ext)
        image = extract_face_region(image)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(dst_dir + file_name + ext, resize(gray))

Now we created a function to perform preprocessing on all images and move them in a separate folder.

In [349]:
preprocess_images(raw_dataset_path + 'angry/', preprocessed_dataset_path + 'angry/')

Processing image: angry_0000.png
Processing image: angry_0001.png
Processing image: angry_0002.png
Processing image: angry_0003.png
Processing image: angry_0004.png
Processing image: angry_0005.png
Processing image: angry_0006.png
Processing image: angry_0007.png


In [350]:
preprocess_images(raw_dataset_path + 'happy/', preprocessed_dataset_path + 'happy/')

Processing image: happy_0000.png
Processing image: happy_0001.png
Processing image: happy_0002.png
Processing image: happy_0003.png
Processing image: happy_0004.png
Processing image: happy_0005.png
Processing image: happy_0006.png
Processing image: happy_0007.png
Processing image: happy_0008.png
Processing image: happy_0009.png
Processing image: happy_0010.png
Processing image: happy_0011.png
Processing image: happy_0012.png
Processing image: happy_0013.png
Processing image: happy_0014.png
Processing image: happy_0015.png
Processing image: happy_0016.png
Processing image: happy_0017.png
Processing image: happy_0018.png


In [351]:
preprocess_images(raw_dataset_path + 'sad/', preprocessed_dataset_path + 'sad/')

Processing image: sad_0000.png
Processing image: sad_0001.png
Processing image: sad_0002.png
Processing image: sad_0003.png
Processing image: sad_0004.png
Processing image: sad_0005.png
Processing image: sad_0006.png
Processing image: sad_0007.png
Processing image: sad_0008.png
Processing image: sad_0009.png
Processing image: sad_0010.png


In [352]:
preprocess_images(raw_dataset_path + 'neutral/', preprocessed_dataset_path + 'neutral/')

Processing image: neutral_0000.png
Processing image: neutral_0001.png
Processing image: neutral_0002.png
Processing image: neutral_0003.png
Processing image: neutral_0004.png
Processing image: neutral_0005.png
Processing image: neutral_0006.png
Processing image: neutral_0007.png
Processing image: neutral_0008.png
Processing image: neutral_0009.png
Processing image: neutral_0010.png
Processing image: neutral_0011.png


### Image Processing

Using low-level processing which takes edges, texture elements or regions for this activity, I will be using Otsu's method to perform thresholding.

In [373]:
def perform_otsu_thresholding(image):
    _, binary_image = cv2.threshold(image, 100, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)
    return binary_image

In [374]:
def threshold_images(src_dir, dst_dir):
    files = os.listdir(src_dir)
    
    for file in files:
        file_name, ext = os.path.splitext(file)
        print("Processing image:", file_name + ext)

        image = cv2.imread(src_dir + file_name + ext)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        res_image = perform_otsu_thresholding(gray_image)
        cv2.imwrite(dst_dir + file_name + ext, res_image)

In [375]:
threshold_images(preprocessed_dataset_path + 'angry/', final_dataset_path + 'angry/')

Processing image: angry_0000.png
Processing image: angry_0001.png
Processing image: angry_0002.png
Processing image: angry_0003.png
Processing image: angry_0004.png
Processing image: angry_0005.png
Processing image: angry_0006.png
Processing image: angry_0007.png


In [376]:
threshold_images(preprocessed_dataset_path + 'happy/', final_dataset_path + 'happy/')

Processing image: happy_0000.png
Processing image: happy_0001.png
Processing image: happy_0002.png
Processing image: happy_0003.png
Processing image: happy_0004.png
Processing image: happy_0005.png
Processing image: happy_0006.png
Processing image: happy_0007.png
Processing image: happy_0008.png
Processing image: happy_0009.png
Processing image: happy_0010.png
Processing image: happy_0011.png
Processing image: happy_0012.png
Processing image: happy_0013.png
Processing image: happy_0014.png
Processing image: happy_0015.png
Processing image: happy_0016.png
Processing image: happy_0017.png
Processing image: happy_0018.png


In [377]:
threshold_images(preprocessed_dataset_path + 'neutral/', final_dataset_path + 'neutral/')

Processing image: neutral_0000.png
Processing image: neutral_0001.png
Processing image: neutral_0002.png
Processing image: neutral_0003.png
Processing image: neutral_0004.png
Processing image: neutral_0005.png
Processing image: neutral_0006.png
Processing image: neutral_0007.png
Processing image: neutral_0008.png
Processing image: neutral_0009.png
Processing image: neutral_0010.png
Processing image: neutral_0011.png


In [378]:
threshold_images(preprocessed_dataset_path + 'sad/', final_dataset_path + 'sad/')

Processing image: sad_0000.png
Processing image: sad_0001.png
Processing image: sad_0002.png
Processing image: sad_0003.png
Processing image: sad_0004.png
Processing image: sad_0005.png
Processing image: sad_0006.png
Processing image: sad_0007.png
Processing image: sad_0008.png
Processing image: sad_0009.png
Processing image: sad_0010.png


## Models

### Training model

I will be using pre-processed images for LBPH Face Recognition and the images that I performed thresholding on will be used for facial recognition for mood detection.

In [1]:
import cv2
import os
import numpy as np

In [8]:
def train_face_recognition_model(data_dir):
    recognizer = cv2.face.LBPHFaceRecognizer_create()

    # Prepare training data
    faces = []
    labels = []

    # Recursively walk through the directory and its subdirectories
    for root, _, files in os.walk(data_dir):
        for filename in files:
            # Check if the file has an image extension
            if any(filename.lower().endswith(ext) for ext in ['.png']):
                # Append the full path of the image file to the list
                faces.append(os.path.join(root, filename))
                labels.append('Ben')

    # Assign unique integer labels to each person
    label_dict = {label: idx for idx, label in enumerate(np.unique(labels))}
    labels = [label_dict[label] for label in labels]

    # Train the recognizer
    recognizer.train(faces, np.array(labels))

    # Save the trained model
    recognizer.save("face_recognition_model.xml")
    print("Face recognition model trained and saved successfully.")

def test_face_recognition_model(image):
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read("face_recognition_model.xml")
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Use LBPH recognizer to predict the identity of the face in the test image
    label, confidence = recognizer.predict(gray_image)

    # If the predicted label matches the specific person you're interested in
    if label == 'Ben':
        print("The specific person's face is recognized.")
    else:
        print("The specific person's face is not recognized.")


In [9]:
train_face_recognition_model(final_dataset_path)

error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'train'
> Overload resolution failed:
>  - Can't parse 'src'. Sequence item with index 0 has a wrong type
>  - Can't parse 'src'. Sequence item with index 0 has a wrong type


## Output